In [1]:
import openpyxl

In [2]:
%run MyFunctions.ipynb

In [3]:
## Import data
filename = "ResearchInChildAndAd_DATA_2018-12-14_1531.xlsx"
workbook = openpyxl.load_workbook(filename)
sheet1 = workbook["ResearchInChildAndAd_DATA_2018-"]
sheet2 = workbook["Tasks"]
sheet3 = workbook["Illness_Keywords"]

In [4]:
## Make columns for new data
IGNORE_KEYWORDS = ['query', 'vs', 'r/o', 'rule out', 'versus']
sheet3_columns = get_column_names(sheet3)
ILLNESS_KEYWORDS = read_in_illness_keywords(sheet3, sheet3_columns)

ILLNESSES = get_key_list(ILLNESS_KEYWORDS)

## Add columns
set_headers(sheet1, 'admission', ILLNESSES)
set_headers(sheet1, 'discharge', ILLNESSES)

In [5]:
## Create a dictionary of column names
column_names = get_column_names(sheet1)

In [6]:
## Module 1: Set binary values to 0
set_binaries_to_zero(sheet1, "admission", ILLNESSES)
set_binaries_to_zero(sheet1, "discharge", ILLNESSES)

In [7]:
## Module 2: Fill in main diagnosis
set_main_diagnosis(sheet1, "admission", IGNORE_KEYWORDS, ILLNESS_KEYWORDS)
set_main_diagnosis(sheet1, "discharge", IGNORE_KEYWORDS, ILLNESS_KEYWORDS)

In [8]:
## Module 3: Read Diagnosis, break into substrings, fill in
one_hot_encode_diagnoses(sheet1, 'admission', IGNORE_KEYWORDS, ILLNESS_KEYWORDS)
one_hot_encode_diagnoses(sheet1, 'discharge', IGNORE_KEYWORDS, ILLNESS_KEYWORDS)

****************************
Ilnesses: ['Depression', 'Intellectual Disability/Global Developmental Delay', 'Suicidal']
Full diagnoses: ['major depressive disorder', 'suicide gesture of undetermined lethality with unclear intent', 'parent child relational disorder', 'query burgeoning personality disorder']
****************************
Ilnesses: ['Depression', 'Intellectual Disability/Global Developmental Delay', 'Suicidal', 'Parent-Child Relational Issues']
Full diagnoses: ['major depressive disorder', 'suicide gesture of undetermined lethality with unclear intent', 'parent child relational disorder', 'query burgeoning personality disorder']
****************************
Ilnesses: ['Depression', 'Intellectual Disability/Global Developmental Delay', 'Suicidal', 'Parent-Child Relational Issues']
Full diagnoses: ['major depressive disorder', 'suicide gesture of undetermined lethality with unclear intent', 'parent child relational disorder', 'query burgeoning personality disorder']
********

****************************
Ilnesses: ['OCD', 'ADHD', 'Intellectual Disability/Global Developmental Delay']
Full diagnoses: ['ocd', 'adhd', 'intellectual disability (mild)', 'parent/child relationship problems']
****************************
Ilnesses: ['OCD', 'ADHD', 'Intellectual Disability/Global Developmental Delay', 'Parent-Child Relational Issues']
Full diagnoses: ['ocd', 'adhd', 'intellectual disability (mild)', 'parent/child relationship problems']
****************************
Ilnesses: ['Depression', 'Disruptive, Impulse-Control, and Conduct', 'Intellectual Disability/Global Developmental Delay', 'Depression']
Full diagnoses: ['unspecified depressive disorder', 'r/o major depressive disorder', 'consider dysthymia', 'unspecified anxiety disorder', 'with general anxiety disorder and social anxiety features', 'r/o panic disorder', ' disruptive behaviour disorder unspecified', 'consider parent child relational issues or odd.']
****************************
Ilnesses: ['Depression', '

****************************
Ilnesses: ['Depression', 'Intellectual Disability/Global Developmental Delay', 'Suicidal']
Full diagnoses: ['major depressive disorder', 'situational crisis acute on chronic suicidality', 'somatic symptom disorder by history', 'eating disorder by history', 'unspecified anxiety disorder by history', 'eating disorder by history', 'query bereavement', '']
****************************
Ilnesses: ['Depression', 'Intellectual Disability/Global Developmental Delay', 'Suicidal', 'Neurodevelopmental', 'Somatic']
Full diagnoses: ['major depressive disorder', 'situational crisis acute on chronic suicidality', 'somatic symptom disorder by history', 'eating disorder by history', 'unspecified anxiety disorder by history', 'eating disorder by history', 'query bereavement', '']
****************************
Ilnesses: ['Depression', 'Intellectual Disability/Global Developmental Delay', 'Suicidal', 'Neurodevelopmental', 'Somatic', 'Eating']
Full diagnoses: ['major depressive d

****************************
Ilnesses: ['Intellectual Disability/Global Developmental Delay', 'Suicidal']
Full diagnoses: ["patient became acutely suicidal and physically aggressive (punched a wall) after finding out about boyfriend's involvement with another woman."]
****************************
Ilnesses: ['OCD', 'Anxiety', 'Disruptive, Impulse-Control, and Conduct', 'Trauma', 'Disruptive, Impulse-Control, and Conduct', 'Disruptive, Impulse-Control, and Conduct', 'Intellectual Disability/Global Developmental Delay']
Full diagnoses: ['ocd', 'other specified anxiety disorder with features of generalized anxiety and social anxiety', 'unspecified trauma and stress-related disorder', 'unspecified disruptive behaviour disorder with features of oppositional defiant disorder', 'mild intellectual disability', 'adhd combined type', 'persistent motor tic disorder', 'parent-child relational problem.']
****************************
Ilnesses: ['OCD', 'Anxiety', 'Disruptive, Impulse-Control, and Cond

****************************
Ilnesses: ['Intellectual Disability/Global Developmental Delay', 'Depression', 'Suicidal']
Full diagnoses: ['major depressive disorder with suicidality and suicide attempt', 'unspecified anxiety', 'rule out anxiety disorder', 'numerous psychosocial stressors and parent-child relational difficulties', '']
****************************
Ilnesses: ['Intellectual Disability/Global Developmental Delay', 'Depression', 'Suicidal', 'Anxiety', 'Disruptive, Impulse-Control, and Conduct']
Full diagnoses: ['major depressive disorder with suicidality and suicide attempt', 'unspecified anxiety', 'rule out anxiety disorder', 'numerous psychosocial stressors and parent-child relational difficulties', '']
****************************
Ilnesses: ['Intellectual Disability/Global Developmental Delay', 'Depression', 'Suicidal', 'Anxiety', 'Disruptive, Impulse-Control, and Conduct']
Full diagnoses: ['major depressive disorder with suicidality and suicide attempt', 'unspecified anxi

In [9]:
## Output file
workbook.save("output-{}.xlsx".format(getTime()))

In [10]:
print(ILLNESSES)
print("***")
print(ILLNESS_KEYWORDS)

['Neurodevelopmental', 'ADHD', 'Intellectual Disability/Global Developmental Delay', 'ASD', 'FASD', 'Schizophrenia Spectrum and Other Psychotic Disorders', 'Bipolar', 'Depression', 'Anxiety', 'OCD', 'Trauma', 'Dissociative', 'Somatic', 'Eating', 'Elimination', 'Sleep-Wake Disorders', 'Gender Dysphoria', 'Disruptive, Impulse-Control, and Conduct', 'Substance-Related and Addictive', 'Personality', 'Suicidal', 'Parent-Child Relational Issues', 'Learning']
***
{'Neurodevelopmental': ['neurodevelopment', 'cerebral', 'palsy', 'chromosom', 'xxx', 'triplex', 'tic', 'prader', 'microdeletion', 'potocki', 'processing', 'neurocognitive', 'tourette'], 'ADHD': ['attention', 'adhd', 'add'], 'Intellectual Disability/Global Developmental Delay': ['intellectual', 'id', 'global', 'gdd', 'delay'], 'ASD': ['autis', 'asd'], 'FASD': ['fasd', 'fetal'], 'Schizophrenia Spectrum and Other Psychotic Disorders': ['schizo', 'psychos', 'psychot', 'delusion', 'catatoni'], 'Bipolar': ['bipolar', 'mani'], 'Depression':